In [ ]:
from collections import namedtuple
import csv
import nltk
import json
# import logging, gensim
# logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

nltk.download('punkt')   # tokenizer
nltk.download('averaged_perceptron_tagger') # POS
nltk.download('wordnet') # similarity

def with_tag(bookmarks, tag):
    return list(filter(lambda item: tag in item.tags, bookmarks))

In [2]:
bookmarks = []
Bookmark = namedtuple('Bookmark', ['title', 'tags'])

with open('marks.csv', newline='', encoding="utf8") as csvfile:
    reader = csv.reader(csvfile, delimiter=',', quotechar='|')
    # transforms lists as strings to real lists
    for row in reader:
        bookmarks.append(Bookmark(row[0], eval(row[1])))
        
for b in bookmarks[30:50]: print(b)

Bookmark(title='Stefan Tosheff - Illustrations', tags=['design', 'people'])
Bookmark(title='\u200eStrange films are strange (actually good)', tags=['film'])
Bookmark(title='OpenGL Reference Card', tags=['compsci', 'graphics'])
Bookmark(title='The Art of Rendering', tags=['compsci', 'graphics'])
Bookmark(title='A Singular Christmas', tags=['music'])
Bookmark(title='List of musical works released in a stem format', tags=['lists', 'music', 'wikipedia'])
Bookmark(title='Alan Ranta', tags=['music', 'people'])
Bookmark(title='James Spectrum Studio', tags=['music'])
Bookmark(title='Tools of the Trade: Zorch - Omnichords', tags=['instruments', 'music'])
Bookmark(title='How to add depth to a mix', tags=['music', 'production'])
Bookmark(title='Chord Theory for Guitar', tags=['music'])
Bookmark(title='List of Internet phenomena', tags=['lists', 'misc', 'wikipedia'])
Bookmark(title='ii-V-I: A Softly Spoken Magic Spell', tags=['music'])
Bookmark(title='Mixtape Alpha with useful links', tags=['music

In [3]:
for b in with_tag(bookmarks, "compsci"):
    print(b)

Bookmark(title='“Right click and save as” needs to go away', tags=['compsci'])
Bookmark(title='Fun with Java2D - Strokes', tags=['compsci'])
Bookmark(title='Books for Computer Science Graduate Students', tags=['compsci'])
Bookmark(title='Chrome Extensions Intro', tags=['compsci'])
Bookmark(title="Google's Python Class", tags=['compsci', 'python'])
Bookmark(title='Java Game Framework', tags=['compsci'])
Bookmark(title='musicalgorithms.ewu.edu', tags=['compsci'])
Bookmark(title='Napier Bones', tags=['compsci'])
Bookmark(title='Unity + XMonad', tags=['compsci'])
Bookmark(title='xmonad - Wikipedia, the free encyclopedia', tags=['compsci'])
Bookmark(title='Java2D examples', tags=['compsci', 'java', 'programming'])
Bookmark(title='OpenGL Reference Card', tags=['compsci', 'graphics'])
Bookmark(title='The Art of Rendering', tags=['compsci', 'graphics'])
Bookmark(title='netcat | computer music', tags=['audiodev', 'awesome', 'compsci', 'music'])
Bookmark(title='music21: a Toolkit for Computer-Ai

In [4]:
all_tags = []
for bookmark in bookmarks:
    all_tags.extend(bookmark.tags)
    
all_tags = list(set(all_tags))
print(all_tags)

['data', 'interviews', 'java', 'webdev', '.sp', 'process', 'c', 'animation', 'private', 'cah', 'blogs', 'art', 'ponte', 'programming', 'people', 'site', 'gallery', 'css', 'functional', '.jobs', 'articles', 'mix', 'dsp', 'production', 'tcc', 'birmingham', 'nix', 'graphics', 'viz', 'compsci', 'film', 'history', 'design', 'cpp', 'js', 'now', 'max', 'music-theory', 'acoustics', 'guitar', 'photography', 'tex', 'lists', 'jazz', 'crypto', 'hardware', 'infosec', 'later', 'videos', '.shopping', 'music', 'tvtropes', 'bmarks', 'proglangs', 'lisp', 'books', '.unlisted', 'typography', 'algorithms', 'awesome', 'ai', 'tweet', 'vim', 'illustration', 'swift', 'installation', 'architecture', 'haskell', 'networks', 'rym', 'misc', 'papers', 'wikipedia', 'haha', 'clojure', 'golang', '.lucio', 'food', 'travel', 'maths', 'interactive', 'electronics', '.campanha', 'academia', 'instruments', '3d', 'gamedev', 'assembly', 'nlp', 'gaming', 'grybo', 'inpe', 'language', '.apto', 'ice', 'sql', '.tab-collection', 'fo

In [5]:
from sklearn.model_selection import train_test_split

bookmarks_train, bookmarks_test = train_test_split(bookmarks, test_size=0.20, random_state=42)

print("Train set:", len(bookmarks_train))
print("Test set: ", len(bookmarks_test))

Train set: 2380
Test set:  595


In [6]:
for bookmark in with_tag(bookmarks_train, "compsci")[:15]:
    tokenized = nltk.word_tokenize(bookmark.title)
    print(nltk.pos_tag(tokenized))

[('tamask', 'NN'), ("'s", 'POS'), ('(', '('), ('KR0', 'NNP'), (')', ')'), ('Gists', 'VBZ')]
[('Framer', 'NNP'), ('-', ':'), ('Animation', 'NN'), ('Prototyping', 'VBG'), ('Tool', 'NN')]
[('Open', 'NNP'), ('Source', 'NNP'), ('Game', 'NNP'), ('Clones', 'NNP')]
[('A', 'DT'), ('few', 'JJ'), ('useful', 'JJ'), ('things', 'NNS'), ('to', 'TO'), ('know', 'VB'), ('about', 'IN'), ('machine', 'NN'), ('learning', 'NN')]
[('You', 'PRP'), ('Might', 'MD'), ('Not', 'RB'), ('Need', 'VB'), ('jQuery', 'NN')]
[('Alzheimer', 'NNP'), ('e', 'NN'), ('Música', 'NNP')]
[('Multiple', 'JJ'), ('lights', 'NNS')]
[('Pac-Man', 'NN'), ('AI', 'NNP')]
[('Latency', 'NN'), ('Numbers', 'NNP'), ('Every', 'NNP'), ('Programmer', 'NNP'), ('Should', 'NNP'), ('Know', 'NNP')]
[('Easing', 'VBG'), ('Equations', 'NNS')]
[('JavaScript', 'NNP'), ('for', 'IN'), ('kids', 'NNS')]
[('Miller', 'NNP'), ('Puckette', 'NNP'), ('MUS171', 'NNP'), ('Videos', 'NNP')]
[('A', 'DT'), ('Course', 'NNP'), ('in', 'IN'), ('Machine', 'NNP'), ('Learning', 'NN

In [7]:
import re
from collections import Counter, defaultdict
from nltk.stem.porter import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
pst = PorterStemmer()
wnl = WordNetLemmatizer()

relevant_pos = [
    "NNP",
    "NN",
    "NNS",
    "JJ",
]

def train(stem_fn):
    tags_stems = defaultdict(lambda: defaultdict(int))
    stems_tags = defaultdict(lambda: defaultdict(int))
    words = []

    for bookmark in bookmarks_train:
        tokenized = nltk.word_tokenize(bookmark.title)
        for word, pos in nltk.pos_tag(tokenized):
            if pos in relevant_pos:
                stem = stem_fn(word.lower())
                if re.match(r"^[\w\d.-]+$", stem):
                    for tag in bookmark.tags:
                        tags_stems[tag][stem] += 1
                        stems_tags[stem][tag] += 1
                    words.append(stem)
                    
    return (tags_stems, stems_tags)
    
# tags_stems, stems_tags = train(porter_stemmer.stem)
tags_stems, stems_tags = train(pst.stem)

In [8]:
for tag in tags_stems:
    print(tag)
    print(sorted(tags_stems[tag].items(), key=lambda k_v: k_v[1], reverse=True))
    print()

data
[('graphql', 2), ('data', 1), ('guid', 1), ('portal', 1), ('facebook', 1), ('dado', 1), ('googl', 1), ('aberto', 1), ('practic', 1), ('brasileiro', 1)]

bmarks
[('learn', 1), ('pinboard', 1), ('featur', 1), ('engin', 1), ('le', 1), ('cognit', 1), ('queri', 1), ('tab', 1), ('misérabl', 1), ('human', 1), ('pagedown', 1), ('gensim', 1), ('sphinx', 1), ('doc', 1), ('year', 1), ('markdown', 1), ('space', 1), ('mutat', 1), ('wysiwyg', 1), ('worth', 1), ('co-occurr', 1), ('topic', 1), ('googl', 1), ('open', 1)]

interviews
[('interview', 16), ('comput', 3), ('alan', 2), ('com', 2), ('donald', 2), ('knuth', 2), ('deerhoof', 2), ('aphex', 2), ('twin', 2), ('kay', 2), ('tatsuya', 1), ('innov', 1), ('secret', 1), ('keenan', 1), ('anyth', 1), ('valu', 1), ('pram', 1), ('nient', 1), ('takahashi', 1), ('bibl', 1), ('vintag', 1), ('orchestra', 1), ('fennesz', 1), ('korg', 1), ('marc-antoin', 1), ('ideia', 1), ('speak', 1), ('sound', 1), ('loren', 1), ('beyond', 1), ('mac', 1), ('alic', 1), ('ent

In [9]:
for stem in stems_tags.items():
    print(json.dumps(stem))

["citi", {"private": 1, "viz": 1, "architecture": 1, "books": 2, "design": 1, "videos": 1, "music": 1, "photography": 3, "tweet": 1, "travel": 2, "birmingham": 1, "programming": 1, "gamedev": 1, "synth": 1, "articles": 1, "misc": 1, "illustration": 1}]
["macbook", {"later": 1}]
["famou", {"crypto": 1, "music": 1, "viz": 1, "compsci": 1, "awesome": 1}]
["learner", {"misc": 1}]
["static", {"graphics": 1, "gamedev": 1, "tweet": 1, "webdev": 1}]
["atipo", {"typography": 1, "foundry": 1}]
["deep", {"nlp": 1, "clojure": 1, "webdev": 1, "music": 1, "awesome": 1, "ai": 6, "art": 1, "tcc": 1, "audiodev": 1, "programming": 2, "synth": 1, "maths": 1, "compsci": 6}]
["mencken", {"books": 1}]
["fretboard", {"music": 1, "programming": 1}]
["oldschool", {"typography": 1, "compsci": 1}]
["stock", {"design": 1}]
["vintag", {"gallery": 1, "interviews": 1, "history": 1, "guitar": 1, "electronics": 2, ".shopping": 1, "videos": 1, "awesome": 2, "production": 2, "compsci": 1, "music": 5, "instruments": 2, "

["mt", {"design": 1}]
["theorem", {"film": 1, "maths": 2, "viz": 1, "compsci": 2, "audiodev": 1}]
["glass", {"music": 1}]
["kaldi", {"tcc": 1}]
["marc-antoin", {"graphics": 1, "people": 2, "interviews": 1, "awesome": 1, "illustration": 2}]
["instagram", {"music": 1, "articles": 1, "people": 1, "compsci": 1, "audiodev": 1}]
["headlines", {"wikipedia": 1, "misc": 1}]
["pointer", {"c": 1, "cpp": 1, "programming": 1}]
["backbone.j", {"compsci": 1, "webdev": 1}]
["summertim", {"music": 1}]
["chorinho", {"music": 1, "private": 1, "grybo": 1}]
["truth", {"books": 1}]
["betterworldbooks.com", {"books": 1}]
["storyboard", {"film": 1, "design": 1, "illustration": 1}]
["helper", {"private": 1, "birmingham": 1}]
["ursula", {"books": 1}]
["eight", {"viz": 1, "interactive": 1}]
["decora\u00e7\u00e3o", {".apto": 1}]
["pram", {"music": 1, "interviews": 1}]
["clara", {"design": 1, "people": 1, "webdev": 1}]
["sine", {"tweet": 1, "maths": 1, "dsp": 1, "audiodev": 2, "haha": 1}]
["analyt", {"webdev": 1}]

["dribbl", {"design": 1, "gallery": 1}]
["platform", {"compsci": 1, "webdev": 1}]
["analysi", {"music-theory": 5, "graphics": 1, "music": 6, "awesome": 1, "later": 1, "papers": 1, "gamedev": 1, "ice": 1, "videos": 1, "compsci": 2, "gaming": 1}]
["manual", {"synth": 1}]
["free", {"css": 1, "3d": 1, "haha": 1, "design": 6, "python": 1, "lists": 2, "gaming": 1, "production": 3, "tweet": 1, "music": 3, "instruments": 1, "audiodev": 1, "webdev": 1, "typography": 2, ".jobs": 1, "articles": 1, "compsci": 3, "misc": 1}]
["stop", {"wikipedia": 1}]
["norma", {"private": 1, "tcc": 1, "misc": 1}]
["monti", {"maths": 1, "wikipedia": 1, "compsci": 1}]
["voxengo", {"lists": 1, "music": 1, "production": 1}]
["harvard", {"algorithms": 1, "ai": 1, "programming": 1, "videos": 1, "compsci": 2, "awesome": 1}]
["rsm-5", {".shopping": 1, "production": 1}]
["bizon", {"music": 1}]
["corr\u00eaa", {"books": 1, "tweet": 1, "illustration": 1}]
["kit", {"music": 2, "instruments": 1, "python": 1, "compsci": 1, "mis

["nadi", {".shopping": 1, "production": 1}]
["filesystem", {"compsci": 1, "programming": 1}]
["witten", {"graphics": 1, "webdev": 1, "people": 1, "audiodev": 1, "programming": 1}]
["co-optimu", {"gaming": 1}]
["properti", {"maths": 1}]
["black", {"tcc": 1, "people": 1}]
["achei", {"blogs": 1, "language": 1, "haha": 1}]
["mine", {"algorithms": 1, "compsci": 2}]
["valu", {"music": 1, "private": 2, "interviews": 1, "programming": 2, ".jobs": 2, "webdev": 2, "process": 1}]
["represent", {"maths": 1, "viz": 1, "haha": 1, "articles": 1, "tcc": 1, "awesome": 1}]
["s.", {"academia": 1, "people": 1, "graphics": 1}]
["meek", {"music": 1, "articles": 1, "production": 1}]
["rivera", {"art": 1, "misc": 1}]
["vidal", {"design": 1, "people": 1, "illustration": 1}]
["ambigram", {"compsci": 1}]
["titl", {"articles": 1, "programming": 1}]
["typographi", {"ice": 1, "film": 3, "gallery": 1, "typography": 5, "design": 11, "people": 1, "compsci": 2, "awesome": 1, "tex": 1}]
["lima", {".jobs": 1}]
["strict",

In [10]:
import itertools

def autotag(title, tags_stems, stem_fn):
    stems = []
    tokenized = nltk.word_tokenize(title)
    for word, pos in nltk.pos_tag(tokenized):
        if pos in relevant_pos:
            stem = stem_fn(word)
            if re.match(r"^[\w\d.-]+$", stem):
                stems.append(stem)
    
    tags_points = defaultdict(int)
    
    for stem in stems:
        for tag in tags_stems:
            for t_stem, count in tags_stems[tag].items():
                if stem == t_stem:
                    tags_points[tag] += count
                    
    return list(itertools.islice(sorted(tags_points.items(), key=lambda kv: -kv[1]), 3))

In [11]:
stem_fn = pst.stem
tags_stems, stems_tags = train(stem_fn)

for b in bookmarks_test:
    print(b.title)
    print("Computed: ", autotag(b.title, tags_stems, pst.stem))
    print("Real tags:", b.tags)
    print()

300 discos importantes da música brasileira
Computed:  [('music', 11), ('lists', 4), ('articles', 2)]
Real tags: ['lists', 'music']

The cinematography of films and TV shows
Computed:  [('film', 25), ('lists', 9), ('animation', 3)]
Real tags: ['colors', 'film']

woscilloscope
Computed:  []
Real tags: ['audiodev', 'music', 'viz']

Academic Earth - Free video lectures
Computed:  [('music', 9), ('design', 9), ('gaming', 8)]
Real tags: ['compsci', 'videos']

Five Worlds
Computed:  [('compsci', 4), ('misc', 4), ('books', 3)]
Real tags: ['articles', 'misc', 'programming']

99% Invisible Podcast - Sounds
Computed:  [('music', 25), ('audiodev', 15), ('production', 11)]
Real tags: ['music', 'production']

Soundslice Tech Talk
Computed:  [('people', 4), ('design', 3), ('videos', 2)]
Real tags: ['compsci']

Principle of charity
Computed:  [('design', 5), ('animation', 2), ('lists', 1)]
Real tags: ['philosophy']

Matthew Butterick
Computed:  [('typography', 1), ('design', 1), ('awesome', 1)]
Real 

Computed:  [('compsci', 6), ('design', 6), ('misc', 5)]
Real tags: ['compsci']

The Sound of Eye
Computed:  [('music', 25), ('audiodev', 15), ('production', 11)]
Real tags: ['blogs', 'film', 'music']

Ping Tunnel - Send TCP traffic over ICMP
Computed:  []
Real tags: ['networks']

How to create 2D visibility/shadow effects for your game
Computed:  [('gaming', 38), ('gamedev', 23), ('compsci', 23)]
Real tags: ['compsci']

Matrix name conventions
Computed:  [('haha', 3), ('lists', 3), ('misc', 2)]
Real tags: ['compsci', 'programming']

No more marinara
Computed:  []
Real tags: ['haha', 'tweet']

Spurious Splines · inconvergent
Computed:  []
Real tags: ['art', 'compsci', 'graphics', 'programming', 'viz']

Shadow DOM — Acko.net
Computed:  [('graphics', 1), ('art', 1), ('audiodev', 1)]
Real tags: ['articles', 'webdev']

Calque
Computed:  []
Real tags: ['language', 'misc', 'wikipedia']

Unistats
Computed:  []
Real tags: ['birmingham', 'private']

Bricasti Design - M7 Reverb Box
Computed:  [('

Computed:  [('compsci', 25), ('ai', 14), ('programming', 7)]
Real tags: ['compsci']

Video games as an art form
Computed:  [('gaming', 48), ('compsci', 30), ('design', 28)]
Real tags: ['art', 'gaming', 'wikipedia']

Careers - SoundHound Inc.
Computed:  [('.jobs', 1), ('private', 1), ('programming', 1)]
Real tags: ['.jobs']

Marcus Lyon
Computed:  []
Real tags: ['people', 'photography']

Subreddit connections
Computed:  [('process', 1), ('gamedev', 1), ('webdev', 1)]
Real tags: ['misc']

‎Movies I Want to Watch Just Because I Like the Cover
Computed:  [('design', 9), ('illustration', 5), ('books', 4)]
Real tags: ['design', 'film', 'lists']

Better web typography: font variants
Computed:  [('design', 29), ('typography', 26), ('webdev', 11)]
Real tags: ['webdev']

demofox / The blog at the bottom of the sea
Computed:  [('compsci', 6), ('blogs', 5), ('programming', 4)]
Real tags: ['audiodev', 'blogs', 'graphics', 'people', 'programming']

Breakthrough films
Computed:  [('film', 24), ('list

Computed:  [('books', 1), ('gaming', 1), ('history', 1)]
Real tags: ['design']

Iosevka - Programming font
Computed:  [('typography', 19), ('design', 14), ('programming', 3)]
Real tags: ['programming', 'typography']

KangHee Kim - Surrealist montages
Computed:  [('design', 1), ('people', 1), ('compsci', 1)]
Real tags: ['gallery', 'people', 'photography']

Machine Visions - Wes Anderson's films visualized
Computed:  [('film', 25), ('lists', 10), ('compsci', 8)]
Real tags: ['film', 'viz']

MIT 6.001 Structure and Interpretation, 1986
Computed:  [('compsci', 9), ('algorithms', 4), ('awesome', 3)]
Real tags: ['algorithms', 'compsci', 'later', 'videos']

Designing with Microcontrollers Final Projects
Computed:  [('design', 4), ('academia', 3), ('audiodev', 3)]
Real tags: ['academia', 'compsci']

Bart Hopkin - Instrument inventor / designer
Computed:  [('design', 56), ('music', 17), ('instruments', 15)]
Real tags: ['instruments', 'music', 'people']

Google Interview Books
Computed:  [('books

Computed:  [('art', 2), ('design', 2), ('people', 2)]
Real tags: ['design', 'people']

Get files inside given directory
Computed:  [('cpp', 2), ('audiodev', 2), ('haha', 2)]
Real tags: ['compsci', 'ice']

Studio Headphones
Computed:  [('production', 8), ('music', 7), ('design', 3)]
Real tags: ['music']

Retro Synth Ads
Computed:  [('synth', 11), ('music', 8), ('design', 6)]
Real tags: ['design', 'illustration', 'synth']

Spurious Correlations
Computed:  [('tcc', 2), ('articles', 1), ('awesome', 1)]
Real tags: ['compsci']

An Illustrated Book of Bad Arguments
Computed:  [('books', 30), ('illustration', 17), ('design', 16)]
Real tags: ['books', 'illustration', 'misc']

DIY USB MIDI controller MPC style
Computed:  [('design', 5), ('programming', 5), ('audiodev', 3)]
Real tags: ['grybo']

Mikhail Pakhomov – Reels and Portfolio
Computed:  [('design', 2), ('people', 2), ('art', 1)]
Real tags: ['animation', 'illustration', 'people']

Programming Languages - Hyperpolyglot
Computed:  [('compsci

Computed:  [('audiodev', 4), ('dsp', 2), ('lists', 1)]
Real tags: ['audiodev', 'compsci', 'music']

Hacker's Guide to Silicon Valley
Computed:  [('compsci', 24), ('design', 9), ('programming', 6)]
Real tags: ['compsci']

CompoHub - Gaming Compos
Computed:  []
Real tags: ['compsci']

Definition of music
Computed:  [('music', 62), ('audiodev', 28), ('compsci', 12)]
Real tags: ['wikipedia']

Mixtape Alpha with useful links
Computed:  [('design', 3), ('compsci', 3), ('tcc', 2)]
Real tags: ['music']

Shadertoy
Computed:  [('graphics', 4), ('awesome', 1), ('programming', 1)]
Real tags: ['compsci', 'graphics']

Almost surely
Computed:  []
Real tags: ['compsci', 'maths', 'wikipedia']

Jason Blevins
Computed:  [('music', 1), ('audiodev', 1), ('programming', 1)]
Real tags: ['people', 'programming']

Grid Case Study
Computed:  [('webdev', 4), ('design', 3), ('misc', 2)]
Real tags: ['awesome', 'css']

Google Visual Assets Guidelines
Computed:  [('compsci', 15), ('viz', 14), ('design', 11)]
Real ta

Computed:  []
Real tags: ['blogs', 'music']

Designing a RESTful Web API with Flask
Computed:  [('webdev', 18), ('compsci', 9), ('python', 8)]
Real tags: ['compsci', 'webdev']

Alan Turing’s Reading List
Computed:  [('lists', 23), ('wikipedia', 18), ('compsci', 12)]
Real tags: ['books']

Fonts - Mark Simonson
Computed:  [('typography', 19), ('design', 15), ('compsci', 4)]
Real tags: ['foundry', 'typography']

A Dweller on Two Planets
Computed:  [('film', 1)]
Real tags: ['books']

libaudiodecoder: The Cross-Platform Audio Decoder API
Computed:  [('audiodev', 21), ('compsci', 10), ('webdev', 8)]
Real tags: ['audiodev', 'c', 'cpp']

Speed Design with Illustrator
Computed:  [('design', 61), ('people', 17), ('illustration', 17)]
Real tags: ['design']

Licenciamento
Computed:  [('inpe', 1), ('.lucio', 1)]
Real tags: ['.lucio', 'inpe']

Ask HN: What tasks do you automate?
Computed:  [('programming', 2), ('music', 1), ('tcc', 1)]
Real tags: ['compsci', 'programming']

Melhores jogos point and 

Computed:  [('design', 4), ('awesome', 3), ('audiodev', 3)]
Real tags: ['instruments', 'music', 'synth']

IDEA - if IKEA designed algorithms
Computed:  [('haha', 2), ('videos', 2), ('compsci', 2)]
Real tags: ['algorithms', 'illustration']

Moon | Typeset In The Future
Computed:  [('design', 4), ('awesome', 2), ('ux', 2)]
Real tags: ['design', 'film', 'typography']

Unity 2D Game Tutorial
Computed:  [('gaming', 38), ('compsci', 32), ('gamedev', 24)]
Real tags: ['gamedev']

macOS Human Interface Guidelines: App Styles and Anatomy
Computed:  [('design', 11), ('compsci', 9), ('ux', 8)]
Real tags: ['design', 'later']

The Pannini Projection
Computed:  [('design', 4), ('webdev', 2), ('audiodev', 2)]
Real tags: ['compsci']

Hydrophones :: H1a
Computed:  []
Real tags: ['music', 'production']

Berklee Alumni Free Samples
Computed:  [('design', 6), ('music', 5), ('audiodev', 3)]
Real tags: ['music', 'production']

NASA - Visions of the Future
Computed:  [('design', 4), ('awesome', 2), ('ux', 2)]

In [12]:
# check if best tag is in real tagset

success = 0

for b in bookmarks_test:
    guess = autotag(b.title, tags_stems, stem_fn)
    if guess and guess[0][0] in b.tags:
        success += 1
    
print(success / len(bookmarks_test))

0.3865546218487395


In [ ]:
# skip if no tags were found

success = 0
skips = 0

for b in bookmarks_test:
    guess = autotag(b.title, tags_stems, stem_fn)
    if not guess:
        skips += 1
        continue
    if guess[0][0] in b.tags: 
            success += 1
    
print(success / (len(bookmarks_test) - skips), skips, "skipped", " / ", len(bookmarks_test))

In [ ]:
# only consider cases where the best tag is above a treshold

success = 0
skips = 0

for b in bookmarks_test:
    guess = autotag(b.title, tags_stems, stem_fn)
    if not guess or guess[0][1] < 10:
        skips += 1
        continue
    if guess[0][0] in b.tags: 
            success += 1
    
print(success / (len(bookmarks_test) - skips), skips, "skipped", " / ", len(bookmarks_test))

In [ ]:
# check if best tag is above a treshold and check if one of the three best tags is in the real tagset

success = 0
skips = 0

for b in bookmarks_test:
    guess = autotag(b.title, tags_stems, stem_fn)
    if not guess or guess[0][1] < 10:
        skips += 1
        continue
    
    for g in guess:
        if g[0] in b.tags:
            success += 1
            break
    
print(success / (len(bookmarks_test) - skips), skips, "skipped", " / ", len(bookmarks_test))

In [ ]:
# same, but using wordnet lemmatizer

success = 0
skips = 0

for b in bookmarks_test:
    guess = autotag(b.title, tags_stems, wnl.lemmatize)
    if not guess or guess[0][1] < 10:
        skips += 1
        continue
    
    for g in guess:
        if g[0] in b.tags:
            success += 1
            break
    
print(success / (len(bookmarks_test) - skips), skips, "skipped", " / ", len(bookmarks_test))

In [ ]:
def autotag2(title, stems_tags, stem_fn):
    stems = []
    tokenized = nltk.word_tokenize(title)
    for word, pos in nltk.pos_tag(tokenized):
        if pos in relevant_pos:
            stem = stem_fn(word)
            if re.match(r"^[\w\d.-]+$", stem):
                stems.append(stem)
    
    tags_points = defaultdict(int)
    
    for stem in stems:
        if stem in stems_tags:
            for tag in stems_tags[stem].items():
                tags_points[tag[0]] += tag[1]
        
    return list(itertools.islice(sorted(tags_points.items(), key=lambda kv: -kv[1]), 3))

In [ ]:
stem_fn = pst.stem
tags_stems, stems_tags = train(stem_fn)

for b in bookmarks_test:
    print(b.title)
    print("Computed: ", autotag2(b.title, stems_tags, pst.stem))
    print("Real tags:", b.tags)
    print()

In [ ]:
# skip if no tags were found

success = 0
skips = 0

for b in bookmarks_test:
    guess = autotag2(b.title, tags_stems, stem_fn)
    if not guess:
        skips += 1
        continue
    if guess[0][0] in b.tags: 
            success += 1
    
print(success / (len(bookmarks_test) - skips), skips, "skipped", " / ", len(bookmarks_test))

In [ ]:
def autotag2(title, stems_tags, stem_fn):
    stems = []
    tokenized = nltk.word_tokenize(title)
    for word, pos in nltk.pos_tag(tokenized):
        if pos in relevant_pos:
            stem = stem_fn(word)
            if re.match(r"^[\w\d.-]+$", stem):
                stems.append(stem)
    
    tags_points = defaultdict(int)
    
    for stem in stems:
        if stem in stems_tags:
            for tag in stems_tags[stem].items():
                tags_points[tag[0]] += tag[1]
        
    return list(itertools.islice(sorted(tags_points.items(), key=lambda kv: -kv[1]), 3))